In [1]:
import os
import json
from gensim.models import LdaModel
from gensim.corpora import Dictionary

In [2]:
corpus = []
docid2fn = []
word2id = {}

# Create corpus

Use each file as a document and each subdomain as a word for LDA.

In [3]:
for fn in sorted(os.listdir('json/')):
    with open('json/' + fn) as f:
        docid2fn.append(fn)
        doc = json.load(f)
        subdomain_freq = {}
        for gene in doc['clusters'][0]['genes']:
            for pfam in gene['pfams']:
                if 'subdomains' in pfam and pfam['subdomains']:
                    subdomain = pfam['subdomains'][0]['name']
                    if subdomain not in word2id:
                        word2id[subdomain] =  len(word2id)
                    subdomainid = word2id[subdomain]
                    if subdomainid in subdomain_freq:
                        subdomain_freq[subdomainid] += 1
                    else:
                        subdomain_freq[subdomainid] = 1
        corpus.append([(k, v) for k,v in subdomain_freq.items()])

In [4]:
dictionary = Dictionary.from_corpus(corpus, {v:k for k,v in word2id.items()})

# Save corpus to disk

In [ ]:
dictionary.save('dict.gensim')
with open('corpus.json', 'w') as f:
    json.dump(corpus, f)
with open('docid2fn.json', 'w') as f:
    json.dump(docid2fn, f)

# Load corpus from disk

In [18]:
dictionary = Dictionary.load('dict.gensim')
with open('corpus.json') as f:
    corpus = json.load(f)
with open('docid2fn.json') as f:
    docid2fn = json.load(f)

# Perform LDA

In [28]:
model = LdaModel(corpus=corpus, num_topics=50, passes=100, id2word=dictionary)

In [29]:
model.show_topics()

[(23,
  '0.096*"PP-binding_c4" + 0.093*"p450_c5" + 0.092*"Condensation_c6" + 0.065*"AMP-binding_c3" + 0.063*"AMP-binding_C_c60" + 0.057*"Condensation_c17" + 0.057*"AMP-binding_c7" + 0.033*"Condensation_c19" + 0.022*"AMP-binding_c12" + 0.020*"PP-binding_c57"'),
 (29,
  '0.106*"Epimerase_c17" + 0.087*"Epimerase_c6" + 0.078*"Glycos_transf_2_c19" + 0.055*"Epimerase_c16" + 0.035*"Condensation_c41" + 0.035*"polyprenyl_synt_c6" + 0.035*"Glycos_transf_2_c14" + 0.028*"Terpene_synth_c6" + 0.021*"Methyltransf_25_c11" + 0.021*"Acetyltransf_1_c40"'),
 (8,
  '0.101*"Acyl-CoA_dh_1_c36" + 0.076*"AMP-binding_c43" + 0.074*"PP-binding_c15" + 0.059*"PP-binding_c9" + 0.056*"ACP_syn_III_c33" + 0.046*"ACP_syn_III_C_c38" + 0.039*"Methyltransf_25_c27" + 0.023*"Aminotran_1_2_c19" + 0.022*"ketoacyl-synt_c27" + 0.019*"AMP-binding_C_c61"'),
 (18,
  '0.141*"Radical_SAM_c26" + 0.068*"B12-binding_c13" + 0.064*"B12-binding_c45" + 0.044*"Glycos_transf_1_c2" + 0.041*"adh_short_c10" + 0.034*"Radical_SAM_c14" + 0.027*"Pep

In [30]:
model.show_topic(1, topn=10)

[('ketoacyl-synt_c11', 0.067112155),
 ('PS-DH_c2', 0.061782002),
 ('ADH_zinc_N_c8', 0.061161797),
 ('p450_c11', 0.048200663),
 ('PP-binding_c7', 0.0388568),
 ('Methyltransf_11_c18', 0.032465268),
 ('p450_c32', 0.032259762),
 ('FAD_binding_3_c3', 0.030505331),
 ('ADH_N_c9', 0.03006145),
 ('Ketoacyl-synt_C_c11', 0.028658535)]

In [39]:
# word freq of a document
doc_id = 0
print(docid2fn[doc_id])
[(dictionary[d[0]], d[1]) for d in corpus[doc_id]]

BGC0000001.json


[('p450_c5', 3),
 ('ACP_syn_III_c39', 1),
 ('ACP_syn_III_C_c35', 1),
 ('PP-binding_c59', 1),
 ('ketoacyl-synt_c27', 1),
 ('Ketoacyl-synt_C_c46', 1),
 ('Acyl_transf_1_c11', 5),
 ('PP-binding_c13', 7),
 ('ketoacyl-synt_c8', 6),
 ('Ketoacyl-synt_C_c2', 6),
 ('PS-DH_c3', 5),
 ('KR_c5', 5),
 ('Acyl_transf_1_c4', 2),
 ('ADH_N_c9', 2),
 ('ADH_zinc_N_c32', 1),
 ('Thioesterase_c22', 1),
 ('ADH_zinc_N_c14', 1)]

In [37]:
# per word topics of a document
_topic_dist, _dum, word_topic = model.get_document_topics(corpus[doc_id], per_word_topics=True)
[(dictionary[d[0]], d[1]) for d in word_topic]

[('p450_c5', [(25, 0.57925737), (37, 2.4207428)]),
 ('ACP_syn_III_c39', [(25, 0.022828491), (37, 0.9771715)]),
 ('ACP_syn_III_C_c35', [(37, 1.0)]),
 ('PP-binding_c59', [(37, 1.0)]),
 ('ketoacyl-synt_c27', [(25, 0.04274818), (37, 0.9572517)]),
 ('Ketoacyl-synt_C_c46', [(25, 0.03726888), (37, 0.9627311)]),
 ('Acyl_transf_1_c11', [(25, 1.3631301), (37, 3.6368704)]),
 ('PP-binding_c13', [(25, 1.1050515), (37, 5.8949485)]),
 ('ketoacyl-synt_c8', [(25, 0.8702633), (37, 5.129737)]),
 ('Ketoacyl-synt_C_c2', [(25, 0.9500488), (37, 5.049951)]),
 ('PS-DH_c3', [(25, 0.860298), (37, 4.139702)]),
 ('KR_c5', [(25, 0.8073037), (37, 4.192696)]),
 ('Acyl_transf_1_c4', [(25, 0.085517325), (37, 1.9144826)]),
 ('ADH_N_c9', [(25, 0.032760445), (37, 1.9672395)]),
 ('ADH_zinc_N_c32', [(25, 0.042736262), (37, 0.95726377)]),
 ('Thioesterase_c22', [(25, 0.17154077), (37, 0.82845926)]),
 ('ADH_zinc_N_c14', [(25, 1.0)])]

# Visualize

In [5]:
import pyLDAvis
import pyLDAvis.gensim

In [5]:
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(model, corpus, dictionary, n_jobs=1)